In [1]:
import numpy as np 
import re 
import requests as req 
import matplotlib.pyplot as plt 
import pandas as pd 
from sodapy import Socrata
import geopy as gp
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import collections as col

In [ ]:
#FOUND API. no need for requests
# requ_url = 'https://www.dot.ny.gov/portal/pls/portal/MEXIS_APP.DYN_PROJECT_DETAILS.show'
# req_post = {'p_arg_names': 'p_pin', 'p_arg_values':801079}
# # resp = req.get(requ_url)
# def data_retrieval(url: str, key: list[str]) -> pd.DataFrame:
#     response = req.get(url)
#     if response.ok == True and response.status_code == 200:
#         print('Success')
#     else:
#         print('Failed')
#     data = response.json()
#     return data
# response = req.get(requ_url, params=req_post)
# print(response.text)

In [2]:
client = Socrata("data.ny.gov",app_token='SUV6CnpKWErb1fdYcxWDEXpHl', username='dwightdinkins@gmail.com',password='AVelaryon@2134')
results = client.get("rz8t-4kmq", limit=5000)
results_df = pd.DataFrame.from_records(results)
results_df.columns

Index(['major_pin', 'contract_number', 'region', 'project_title',
       'project_status', 'status', 'bid_opening_date', 'federal_funding',
       'state_funding', 'local_funding', 'type_of_work',
       'public_friendly_description', 'contract_award_date',
       'contract_award_amount', 'approved_cost_changes',
       'current_award_amount', 'estimated_or_actual_completed_date',
       'schedule_performance', 'cost_performance',
       'in_future_development_start_date', 'construction_start_date',
       'construction_end_date', 'construction_amount'],
      dtype='object')

# Data Pre-processing

In [3]:
group_status = results_df.groupby(by='status', as_index=False)
indx = (results_df.status.str.contains('Completed Project', regex=False).to_numpy() == 1).nonzero()

In [4]:
pd.set_option('display.max_columns', 23)
completed = results_df.iloc[indx[0],:].copy()
# Converted COntract Award Date & Estimated Completion Date to Datetime Objects
completed.contract_award_date = pd.to_datetime(completed.contract_award_date)
completed.estimated_or_actual_completed_date = pd.to_datetime(completed.estimated_or_actual_completed_date)

In [5]:
completed_dropped_contruct = completed.drop(columns=['construction_start_date', 'construction_end_date', 'construction_amount']).copy()

In [ ]:
completed_dropped_contruct

In [6]:

def modify_addy(regex: list[str], data: pd.DataFrame, dat_col: pd.Series):
    geolocator = Nominatim(user_agent='myGeocoder', timeout=10)
    geolocator = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    hwy_data = dat_col.str.replace(regex[0],r'\1-\2', regex=True).str.extract(regex[1]).dropna()
    loc_data = col.defaultdict(dict)
    for indx, hwy in hwy_data.to_dict()[0].items():
        try:
            location = geolocator(hwy + ', NY', geometry='geojson')
            if location != None:
                loc_data['latitude'][indx] = location.latitude
                loc_data['longitude'][indx] = location.longitude
                # print(loc_data['longitude'][indx])
            else:
                loc_data['latitude'][indx] = np.nan
                loc_data['longitude'][indx] = np.nan
        except GeocoderUnavailable:
            pass        

    return loc_data
    
    

In [ ]:
regex = [r'([NYIRtT\.]+)\s?(\d+)',r'([NYIRtT\.]+-\d+)']
location_data = modify_addy(regex, completed_dropped_contruct, completed_dropped_contruct.project_status)

In [ ]:
location_data1 = modify_addy(regex, completed_dropped_contruct, completed_dropped_contruct.project_title)

In [7]:
location_data = pd.read_excel(r"C:\Users\dwigh\Downloads\location_data1.xlsx")

In [8]:
pd.set_option('display.max_rows', 300)
location_data.set_index('Unnamed: 0', inplace=True)
location_data.index.name = ''

In [9]:
pd.set_option('display.max_rows', 300)
regex = r'(\d+\s\w+\s[RAdVEveoOaD]+)'
regex1 = r'(\s[\w\s]+and\s[\w\s]+)'
regex2 = r'(?<=between\s)([\w\s]+)and([\w\s]+)'
regex3 = r'(?<=FROM\s)([\w\d\s]+)'
regex4 = r'([NYIRtT\.]+)\s?(\d+)'
regex5 = r'([NYIRtT\.]+-\d+)'
# completed_dropped_contruct.project_status.str.replace(regex4, r'\1-\2', regex=True).str.extract(regex5).dropna()
completed_dropped_contruct.project_status.str.extract(regex1).dropna().to_dict()
# completed_dropped_contruct.project_status.str.extract(regex1)
# for (i, item) in completed_dropped_contruct.public_friendly_description.to_dict().items():
# completed_dropped_contruct.project_status.to_list()


{0: {0: ' Phase 3 extends between TOBAY and Captree State Park Towns of Oyster Bay',
  20: ' Youngs Hill Road in the towns of Huntington and Babylon Suffolk County',
  41: ' repair or replace lighting systems and appurtenances on various federal aid eligible state highways in Nassau ',
  51: ' deck replacement and repairs on I495 WB',
  63: ' Pavement Restoration including joint sealing and diamond grinding at Ocean Parkway w',
  93: ' Hempstead and Oyster Bay',
  329: ' to construct a bicycle and pedestrian connection from Broadway to the Mohawk Hudson Bike Trail along the Hudson River',
  477: ' Pavement Rehabilitation and Safety Improvement',
  621: ' This project will address Demand Maintenance Repairs in New York City',
  707: ' proposes to install a traffic signal at the intersection of NY 77 and Ledge Rd in the Town of Alabama',
  710: ' Rt 96 from Mitchell Rd to Garnsey Rd in the Towns of Pittsford and Perinton',
  717: ' mill and fill with stress relief pavement repairs on Rt 

In [21]:
rem ={
    0: "Captree State Park Towns of Oyster Bay",
    20: "Youngs Hill Road, Huntington",
    41: "Nassau",
    51: "I-495 ",
    63: "Ocean Parkway ",
    93: "Hempstead & Oyster Bay",
    329: "Hudson River",
    477: 'General',
    621: "New York City",
    707: "Ledge Rd, Alabama",
    710: "Mitchell Rd,Pittsford",
    717: "Pound Road, Williamson",
    718: "Rt-259",
    727: "Lowenthal Road, Henrietta",
    751: 'General',
    803: "Crescent Beach",
    827: 'General',
    885: "Rt-83",
    934: "Erie & Cattaraugus",
    951: 'General',
    952: "South Abbott Rd",
    1484: "Putnam & Westchester",
    1641: "Rt-205",
    1673: 'General',
    1684: "Cornell Creek & Wilkins Brook",
    1698: 'General',
    1699: "I-88",
    1706: "General",
    1708: 'General',
    1710: "Kirkwood & Binghamton",
    1711: "Kirkwood & Binghamton",
    1717: "Delaware & Sullivan ",
    1719: "Broome County & Signal",
    1720: 'General',
    1722: "Otsego & Sullivan Counties",
    1748: 'General',
    1749: 'General',
    1750: 'General',
    1752: 'General',
    1755: "General",
    1787: 'General',
    1796: "Rt",
    1797: "Hartwick & Otsego",
    1798: "Middletown & Margaretville",
    1799: "Richford & Cortland County",
    1816: 'General',
    1820: "Halcottsville & Roxbury",
    1865: 'General',
    1866: "General",
    1867: "Richmond counties",
    1882: 'General',
    1896: 'General',
    1897: "Richmond",
    1900: "Bronx county",
    1904: "NYC",
    1905: "General",
    1908: "BQE East & West bound from Flushing Ave"
}


In [22]:
geolocator = Nominatim(user_agent='myGeocoder', timeout=10)
geolocator = RateLimiter(geolocator.geocode, min_delay_seconds=1)
loc_data = col.defaultdict(dict)
for indx, hwy in rem.items():
    try:
        location = geolocator(hwy + ', NY', geometry='geojson')
        if location != None:
            loc_data['latitude'][indx] = location.latitude
            loc_data['longitude'][indx] = location.longitude
            # print(loc_data['longitude'][indx])
        else:
            loc_data['latitude'][indx] = np.nan
            loc_data['longitude'][indx] = np.nan
    except gp.exc.GeocoderUnavailable:
        pass

In [24]:
loc_data['latitude']


{0: nan,
 20: 40.88912245249046,
 41: 40.7352994,
 51: 40.76020774512672,
 63: 40.576312,
 93: 40.725655661915994,
 329: 42.6072474,
 477: 50.37318615,
 621: 40.7127281,
 707: 43.04548295148877,
 710: 43.084053259962644,
 717: 43.2442606209997,
 718: 42.08312349352062,
 727: 43.090527443619486,
 751: 50.37318615,
 803: 43.2836724,
 827: 50.37318615,
 885: 43.3711998,
 934: 42.75456647782082,
 951: 50.37318615,
 952: 42.714838260514846,
 1484: 41.27727115,
 1641: 41.1223324,
 1673: 50.37318615,
 1684: nan,
 1698: 50.37318615,
 1699: 42.45240237488124,
 1706: 50.37318615,
 1708: 50.37318615,
 1710: 42.1331651,
 1711: 42.1331651,
 1717: 41.761111,
 1719: nan,
 1720: 50.37318615,
 1722: nan,
 1748: 50.37318615,
 1749: 50.37318615,
 1750: 50.37318615,
 1752: 50.37318615,
 1755: 50.37318615,
 1787: 50.37318615,
 1796: 43.1019148,
 1797: 42.6597973,
 1798: 42.146882250000004,
 1799: 42.41791869628468,
 1816: 50.37318615,
 1820: 42.219185867095355,
 1865: 50.37318615,
 1866: 50.37318615,
 1867

In [26]:
location_data.latitude.fillna(value=loc_data['latitude'])


0             NaN
17      40.657336
20      40.689344
21      40.691521
29      40.760208
32            NaN
33            NaN
38      40.760208
41      40.735299
42            NaN
43            NaN
44            NaN
46            NaN
48            NaN
49            NaN
51      40.760208
59            NaN
62            NaN
63      40.576312
64            NaN
68            NaN
93      40.725656
248     42.740265
250     42.740265
254           NaN
279           NaN
280     42.620726
289     40.878576
299     42.740265
300     40.779578
301     43.206267
305     40.650035
307     41.026125
310     41.026125
314     43.206307
327           NaN
329     42.607247
403           NaN
434     43.452652
442           NaN
467     40.688642
472     43.050413
473     43.159193
476     44.983857
477     13.524834
478     43.159193
479     43.182177
480     43.452652
481     13.543015
484     42.093805
485           NaN
487           NaN
488           NaN
490           NaN
491           NaN
497      

In [ ]:
completed_dropped_contruct.project_status.str.replace(r'')

In [19]:
# 1797
geolocator = Nominatim(user_agent='myGeocoder')
location = geolocator.geocode('RT-9a'+', NY', geometry='wkt', exactly_one=True)

In [ ]:
words = completed_dropped_contruct.type_of_work.str.split().to_numpy()
unique_set = list()
for word in words:
  unique_set.extend(word)
num_of_appearance = dict()
for item in set(unique_set):
  num_of_appearance[item] = unique_set.count(item)